In [1]:
import pathlib

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [3]:
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import pytesseract
import tensorflow as tf
import re

print(tf.__version__)

2.1.0


In [4]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [5]:
tf.gfile = tf.io.gfile

utils_ops.tf = tf.compat.v1

vis_util.tf = tf.compat.v1

In [6]:
def load_exist_model(model_name):
    model_dir = model_name
    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [7]:
model_name = "ssd_mobilenet_v1_coco_2018_01_28"
model = load_exist_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [9]:
model.output_dtypes

{'detection_classes': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32,
 'detection_scores': tf.float32}

In [11]:
model.output_shapes

{'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 100, 4]),
 'detection_scores': TensorShape([None, 100])}

In [12]:
category_index[89]['name']

'hair drier'

In [13]:
#비디오에서 적용

capture = cv2.VideoCapture("files/training_video.mp4") # https://github.com/IBM/powerai-counting-cars

while True:
    
    ret, frame = capture.read()
    
    if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        break
        
    image = np.asarray(frame)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]

    output_dict = model(input_tensor)

    # 추론 결과 표시
    rows, cols, _ = image.shape
    for i in range(int(output_dict['num_detections'])):
        class_id = int(output_dict['detection_classes'][0][i])
        score = output_dict['detection_scores'][0][i].numpy()
        box = output_dict['detection_boxes'][0][i]

        if score > 0.5:
            x1 = int(box[1] * cols)
            y1 = int(box[0] * rows)
            x2 = int(box[3] * cols)
            y2 = int(box[2] * rows)


            cv2.putText(image, category_index[class_id]['name'] + ":" + str(score), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX,
                        1.5, (0, 0, 255), 1)
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 5)

    cv2.imshow("Object Detections", image)
    
    if cv2.waitKey(33) == ord('q'): 
        cv2.destroyAllWindows()
        break

In [15]:
# 비디오에서 적용
# 객체의 움직임 표현도 같이 표시

# capture = cv2.VideoCapture("files/bird.mp4") # https://www.youtube.com/watch?v=XdlIbNrki5o
capture = cv2.VideoCapture("files/training_video.mp4") # https://github.com/IBM/powerai-counting-cars


ret, prev_frame = capture.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_pts = cv2.goodFeaturesToTrack(prev_gray, mask = None, maxCorners = 1000, qualityLevel = 0.1,
                                    minDistance = 5, blockSize = 9)
    
while True:
    
    ret, next_frame = capture.read()
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
    frame = next_frame.copy()
    
    if (capture.get(cv2.CAP_PROP_POS_FRAMES) == capture.get(cv2.CAP_PROP_FRAME_COUNT)):
        break
        
    input_img = cv2.resize(frame, (300,300))
    input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
        
    input_img = np.asarray(input_img)
    input_tensor = tf.convert_to_tensor(input_img)
    input_tensor = input_tensor[tf.newaxis,...]

    output_dict = model(input_tensor)

    # 추론 결과 표시
    rows, cols, _ = image.shape
    for i in range(int(output_dict['num_detections'])):
        class_id = int(output_dict['detection_classes'][0][i])
        score = output_dict['detection_scores'][0][i].numpy()
        box = output_dict['detection_boxes'][0][i]

        if score > 0.7:
            x1 = int(box[1] * cols)
            y1 = int(box[0] * rows)
            x2 = int(box[3] * cols)
            y2 = int(box[2] * rows)
            
            # 객체의 움직임 경로 표시
            flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, flow=None, pyr_scale=0.5,
                                                levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.1,
                                                flags = cv2.OPTFLOW_FARNEBACK_GAUSSIAN)

            # 객체의 움직임 표현
            magnitude, angle = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)
            magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
            angle = angle / 2
                
            hsv = np.zeros_like(next_frame)
            hsv[:,:,0] = angle
            hsv[:,:,1] = magnitude
            hsv[:,:,2] = 255
            frame = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
            
            for i in range(x1, x2, 15):
                for j in range(y1, y2, 15):
                    x = int(flow[j, i, 0])
                    y = int(flow[j, i, 1])

                    cv2.circle(frame, (i, j), 1, (0, 0, 255), 1)
                    cv2.line(frame, (i, j), (i + x, j + y), (0, 255, 255), 1)
                    cv2.circle(frame, (i + x, j + y), 1, (0, 255, 255), 1)

            cv2.putText(frame, category_index[class_id]['name'] + ":" + str(score), (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX,
                        1.5, (0, 0, 255), 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 5)

    cv2.imshow("Object Detections", frame)
    
    if cv2.waitKey(33) == ord('q'): 
        cv2.destroyAllWindows()
        break